In [8]:
import cv2
import numpy as np

def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply thresholding to obtain a binary image
    _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY_INV)

    return thresh

def detect_bubbles(image):
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bubbles = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if 10 < area < 200:  # Area filter for bubble size
            # Get the bounding box of the contour
            (x, y, w, h) = cv2.boundingRect(contour)
            bubbles.append((x, y, w, h))

    return bubbles

def sort_bubbles(bubbles):
    # Sort by y coordinate first (row-wise sorting)
    bubbles = sorted(bubbles, key=lambda b: (b[1], b[0]))

    # Group into rows (assume 5 bubbles per question)
    rows = []
    row = []
    for i, bubble in enumerate(bubbles):
        row.append(bubble)
        # Every 5 bubbles, start a new row
        if (i + 1) % 5 == 0:
            # Sort each row by x coordinate (left to right)
            row = sorted(row, key=lambda b: b[0])
            rows.append(row)
            row = []

    return rows

def draw_bubble_numbers(image, rows):
    for i, row in enumerate(rows):
        for j, (x, y, w, h) in enumerate(row):
            # Draw a rectangle around each bubble (optional)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Label each bubble with its question and option number
            text = f"Q{i+1}O{j+1}"
            cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    return image

# Load the image
image = cv2.imread('warped_answer_sheet.png')

# Preprocess the image (grayscale, thresholding)
thresh_image = preprocess_image(image)

# Detect bubbles
bubbles = detect_bubbles(thresh_image)

# Sort and group bubbles by question and option
sorted_bubbles = sort_bubbles(bubbles)

# Draw bubbles with question and option numbers on the image
output_image = draw_bubble_numbers(image.copy(), sorted_bubbles)

# Display the result
cv2.imshow("Detected Bubbles by Question", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Optionally save the result
cv2.imwrite('bubbles_by_question.png', output_image)


True